In [163]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
script_df = pd.read_csv('script.csv')

In [3]:
script_lab = script_df[['cast','dialog']]

In [4]:
#Filter for only "main" characters (easier for classification) based on counts in dataset 
script_cnts = script_lab.groupby('cast').count().sort_values(by='dialog',ascending=False)
script_cnts.head()


,dialog
cast,
Frasier,21088
Niles,10178
Martin,7629
Daphne,5243
Roz,4735


In [5]:
#Do the filtering
min_samples = 1000
main_chars=script_cnts[script_cnts['dialog']>=min_samples].index
script_filt = script_lab[script_lab['cast'].isin(main_chars)].copy()

In [6]:
# fix bracketed actions. 
#script_filt.loc[:,['dialog']] = 
script_filt['dialog'] = script_filt['dialog'].str.replace(("\[.*\]"),'')

In [53]:
print(main_chars)

Index(['Frasier', 'Niles', 'Martin', 'Daphne', 'Roz'], dtype='object', name='cast')


In [27]:
script_filt = script_filt.dropna()

In [164]:
# bag of words 
def get_vector_represet(script_df): 
    """Change a dataframe of cast lines into a matrix representation of those lines, and a matrix representation of who said them."""

    corpus = script_df['dialog']

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    Y = pd.get_dummies(script_df['cast'])
    
    return X,Y 

In [65]:
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=.2,random_state=1930)


In [66]:
num_training, num_features = xtrain.get_shape()

In [159]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(units=800, activation='sigmoid',input_shape=(num_features,)))
#model.add(Dropout(.2,noise_shape=None, seed=None))
#model.add(Dense(units=4200, activation='sigmoid'))
#model.add(Dense(units=500, activation='softmax'))
#model.dropout(.2)
model.add(Dense(units=150, activation='tanh'))
model.add(Dense(units=5, activation='relu'))



In [160]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [161]:
model.fit(xtrain,ytrain,epochs=1,batch_size=32)

Epoch 1/1
39098/39098 [==============================] - 178s 5ms/step - loss: 7.8650 - acc: 0.4303


In [162]:
model.evaluate(xtest,ytest,batch_size=128)

9775/9775 [==============================] - 8s 791us/step


[7.802059611640013, 0.43508951413357044]

In [95]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 5000)              101285000 
_________________________________________________________________
dense_40 (Dense)             (None, 780)               3900780   
_________________________________________________________________
dense_41 (Dense)             (None, 120)               93720     
_________________________________________________________________
dense_42 (Dense)             (None, 5)                 605       
Total params: 105,280,105
Trainable params: 105,280,105
Non-trainable params: 0
_________________________________________________________________
